In [1]:
import pandas as pd
ks=pd.read_csv('G:/MINE/SUB/Thesis/kaggle/6.Feature Engineering/DataSet/ks-projects-201801.csv',parse_dates=['deadline','launched'])
ks=ks.query("state!='live'")
ks=ks.assign(outcome=(ks.state=='successful').astype(int))
ks=ks.assign(hour=ks.launched.dt.hour,
            day=ks.launched.dt.day,
            month=ks.launched.dt.month,
            year= ks.launched.dt.year)
num_col=['goal','hour','day','month','year','outcome']
cat_col=['category','currency','country']
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
encode=ks[cat_col].apply(encoder.fit_transform)
baseline=ks[num_col].join(encode)
import itertools
interaction=pd.DataFrame(index=ks.index)
for col1,col2 in itertools.combinations(cat_col,2):
    col_name=col1+'_'+col2
    col_value=ks[col1].map(str)+'_'+ks[col2].map(str)
    label_encoder=LabelEncoder()
    interaction[col_name]=label_encoder.fit_transform(col_value)

interBaseline=baseline.join(interaction)
interBaseline.head()
release=pd.Series(ks.index,index=ks.launched,name='count_7days').sort_index()
count_7days=release.rolling('7d').count()-1
count_7days.index=release.values
count_7days=count_7days.reindex(ks.index)
cout_inter_baseline=interBaseline.join(count_7days)
sameProject=pd.DataFrame(ks[['category','launched']],index=ks.index).sort_values('launched')
def time_difference(series):
    return series.diff().dt.total_seconds()/3600
timedeltas=sameProject.groupby('category').transform(time_difference)
timedeltas=timedeltas.fillna(timedeltas.max())
final_baseline=cout_inter_baseline.join(timedeltas.rename({'launched':'time_since_last_Project'},axis=1))
def make_split(df,fraction=.1):
    size=int(len(df)*fraction)
    train=df[:-2*size]
    valid=df[-2*size:-size]
    test=df[-size:]
    return train,valid,test

def train_model(train,valid,test=None,feature=None):
    import lightgbm as lgb
    from sklearn.metrics import roc_auc_score
    if feature is None:
        feature=train.columns.drop('outcome')
    dtrain=lgb.Dataset(train[feature],label=train['outcome'])
    dvalid=lgb.Dataset(valid[feature],label=valid['outcome'])
    param={'num_leaves':64,'objective':'binary','metric':'auc','seed':7}
    num_round=1000
    model=lgb.train(param,dtrain,num_round,valid_sets=[dvalid],verbose_eval=False,early_stopping_rounds=20)
    validPredict=model.predict(valid[feature])
    print('\n'*4)
    print("wait a second to see accuracy on valid sets .. .")
    print("valid accuracy = {}".format(roc_auc_score(valid['outcome'],validPredict)))
    if test is not None:
        print('wait another second to see accuracy on test set .. .')
        print('test accuracy = {}'.format(roc_auc_score(test['outcome'],model.predict(test[feature]))))

In [2]:
train,valid,test=make_split(final_baseline)
train_model(train,valid,test)

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020325 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1553
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501





wait a second to see accuracy on valid sets .. .
valid accuracy = 0.7466774415346332
wait another second to see accuracy on test set .. .
test accuracy = 0.746659055391359


In [4]:
from sklearn.feature_selection import SelectKBest,f_classif #ANOVA f value 
feature_col=final_baseline.columns.drop('outcome')
train,valid,test=make_split(final_baseline)
selector=SelectKBest(f_classif,k=5)
x_uni=selector.fit_transform(train[feature_col],train['outcome'])
x_uni

array([[2.015e+03, 5.000e+00, 9.000e+00, 1.800e+01, 1.409e+03],
       [2.017e+03, 1.300e+01, 2.200e+01, 3.100e+01, 9.570e+02],
       [2.013e+03, 1.300e+01, 2.200e+01, 3.100e+01, 7.390e+02],
       ...,
       [2.011e+03, 1.300e+01, 2.200e+01, 3.100e+01, 5.150e+02],
       [2.015e+03, 1.000e+00, 3.000e+00, 2.000e+00, 1.306e+03],
       [2.013e+03, 1.300e+01, 2.200e+01, 3.100e+01, 1.084e+03]])

In [5]:
select_feature=pd.DataFrame(selector.inverse_transform(x_uni),index=train.index,columns=feature_col)
select_feature.head(7)

,goal,hour,day,month,year,category,currency,country,category_currency,category_country,currency_country,count_7days,time_since_last_Project
0,0.0,0.0,0.0,0.0,2015.0,0.0,5.0,9.0,0.0,0.0,18.0,1409.0,0.0
1,0.0,0.0,0.0,0.0,2017.0,0.0,13.0,22.0,0.0,0.0,31.0,957.0,0.0
2,0.0,0.0,0.0,0.0,2013.0,0.0,13.0,22.0,0.0,0.0,31.0,739.0,0.0
3,0.0,0.0,0.0,0.0,2012.0,0.0,13.0,22.0,0.0,0.0,31.0,907.0,0.0
4,0.0,0.0,0.0,0.0,2015.0,0.0,13.0,22.0,0.0,0.0,31.0,1429.0,0.0
5,0.0,0.0,0.0,0.0,2016.0,0.0,13.0,22.0,0.0,0.0,31.0,1284.0,0.0
6,0.0,0.0,0.0,0.0,2014.0,0.0,13.0,22.0,0.0,0.0,31.0,1119.0,0.0


In [7]:
selected_col=select_feature.columns[select_feature.var()!=0]
selected_col

Index(['year', 'currency', 'country', 'currency_country', 'count_7days'], dtype='object')

In [12]:
selected_col=list(selected_col)


In [15]:
selected_col.append('outcome')

In [16]:
(selected_col)

['year', 'currency', 'country', 'currency_country', 'count_7days', 'outcome']

In [17]:
uni_train=train[selected_col]
uni_valid=valid[selected_col]
uni_test=test[selected_col]

In [18]:
train_model(uni_train,uni_valid,uni_test)

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012828 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 335
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501





wait a second to see accuracy on valid sets .. .
valid accuracy = 0.6010188249130639
wait another second to see accuracy on test set .. .
test accuracy = 0.6038882667359431


In [19]:
help(SelectKBest)

Help on class SelectKBest in module sklearn.feature_selection._univariate_selection:

class SelectKBest(_BaseFilter)
 |  SelectKBest(score_func=<function f_classif at 0x0000020ABDB63790>, *, k=10)
 |  
 |  Select features according to the k highest scores.
 |  
 |  Read more in the :ref:`User Guide <univariate_feature_selection>`.
 |  
 |  Parameters
 |  ----------
 |  score_func : callable
 |      Function taking two arrays X and y, and returning a pair of arrays
 |      (scores, pvalues) or a single array with scores.
 |      Default is f_classif (see below "See also"). The default function only
 |      works with classification tasks.
 |  
 |      .. versionadded:: 0.18
 |  
 |  k : int or "all", optional, default=10
 |      Number of top features to select.
 |      The "all" option bypasses selection, for use in a parameter search.
 |  
 |  Attributes
 |  ----------
 |  scores_ : array-like of shape (n_features,)
 |      Scores of features.
 |  
 |  pvalues_ : array-like of shape (